In [1]:
import openeo

In [2]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

## Batch jobs basics

Let's do a simple batch job first: compute pixel-wise means over an area.

1. Specify the range of raw data that need to be loaded by the server for further processing.

In [3]:
period=("2022-05-01", "2022-06-01")
aoi={
        "west": 3.20,
        "south": 51.18,
        "east": 3.70,
        "north": 51.68,
        "crs": "EPSG:4326",
    }

In [4]:
# Server-side computations with Sentinel 5 data process one band at a time
s5_CH4 = connection.load_collection(
    "SENTINEL_5P_L2",
    spatial_extent=aoi,
    temporal_extent = period,
    bands=["CH4"]
)

In [5]:
# Server-side computations with Sentinel 3 data process several bands at a time
s3_all_bands = connection.load_collection(
    "SENTINEL3_SLSTR",
    spatial_extent=aoi,
    temporal_extent = period,
    bands=["S5", "S6"]
)

2. Define the server-side computations that need to be performed on the raw data. For now, we are just computing pixel-wise means. We will make this more advanced in a moment.

In [6]:
s5_CH4_mean = s5_CH4.reduce_dimension(dimension="t", reducer="mean")
s5_CH4_mean = s5_CH4_mean.save_result(format="JSON")

In [7]:
s3_mean = s3_all_bands.reduce_dimension(dimension="t", reducer="mean")
s3_mean = s3_mean.save_result(format="JSON")

3. Create a batch job. 

In [8]:
job=s3_mean.create_job(title="Mean-S3-aoi")

In [9]:
job = s5_CH4_mean.create_job(title="Mean-S5-CH4-aoi")

Batch jobs allow you to start a job, leave (and shut down your device), let it run on the servers and then reconnect later. For that, you need to know the job_id.

In [10]:
job_id=job.job_id

Start the job. 

In [11]:
job.start_and_wait()

0:00:00 Job 'j-240212870c1b45d2a7a1018805c6a54e': send 'start'
0:00:16 Job 'j-240212870c1b45d2a7a1018805c6a54e': created (progress N/A)
0:00:21 Job 'j-240212870c1b45d2a7a1018805c6a54e': created (progress N/A)
0:00:28 Job 'j-240212870c1b45d2a7a1018805c6a54e': created (progress N/A)
0:00:36 Job 'j-240212870c1b45d2a7a1018805c6a54e': created (progress N/A)
0:00:47 Job 'j-240212870c1b45d2a7a1018805c6a54e': running (progress N/A)
0:01:00 Job 'j-240212870c1b45d2a7a1018805c6a54e': running (progress N/A)
0:01:17 Job 'j-240212870c1b45d2a7a1018805c6a54e': running (progress N/A)
0:01:38 Job 'j-240212870c1b45d2a7a1018805c6a54e': running (progress N/A)
0:02:02 Job 'j-240212870c1b45d2a7a1018805c6a54e': running (progress N/A)
0:02:33 Job 'j-240212870c1b45d2a7a1018805c6a54e': running (progress N/A)
0:03:11 Job 'j-240212870c1b45d2a7a1018805c6a54e': finished (progress N/A)


<BatchJob job_id='j-240212870c1b45d2a7a1018805c6a54e'>

Note that execute_batch() can both create and start the job for you but it is not suitable for long-running jobs, because you will need to keep your client running all the time.
To reconnect to a job at a later time, do

In [12]:
job = connection.job(job_id)

Useful commands for exploring your jobs:

In [13]:
# list your batch jobs
connection.list_jobs()
# check the status of the job
job.status()
# check the logs
job.logs()
# cancel a job
job.stop()
# see job details, including the cost ( see monthly quotas https://documentation.dataspace.copernicus.eu/Quotas.html) and a visual representation of how the job was constructed
job 
# job meta data
job.describe()

{'costs': 3.0,
 'created': '2024-02-12T23:43:32Z',
 'id': 'j-240212870c1b45d2a7a1018805c6a54e',
 'process': {'process_graph': {'loadcollection1': {'arguments': {'bands': ['CH4'],
     'id': 'SENTINEL_5P_L2',
     'spatial_extent': {'crs': 'EPSG:4326',
      'east': 3.7,
      'north': 51.68,
      'south': 51.18,
      'west': 3.2},
     'temporal_extent': ['2022-05-01', '2022-06-01']},
    'process_id': 'load_collection'},
   'reducedimension1': {'arguments': {'data': {'from_node': 'loadcollection1'},
     'dimension': 't',
     'reducer': {'process_graph': {'mean1': {'arguments': {'data': {'from_parameter': 'data'}},
        'process_id': 'mean',
        'result': True}}}},
    'process_id': 'reduce_dimension'},
   'saveresult1': {'arguments': {'data': {'from_node': 'reducedimension1'},
     'format': 'JSON',
     'options': {}},
    'process_id': 'save_result',
    'result': True}}},
 'status': 'finished',
 'title': 'Mean-S5-CH4-aoi',
 'updated': '2024-02-12T23:46:15Z',
 'usage': {'

Download the results

In [14]:
PATH="output"
results = job.get_results()
results #to get the properties of the results
#results.download_files(PATH) #to download all files (including json with job metadata)
results.download_file("output/results.json") #to download a specific file

PosixPath('output/results.json')

## Combine locations and compute mean per area of interest for a given timestamp

We are going to use the function aggregate_spatial(). It takes a list of areas of interest and compute the statistics per area (not pixel-wise as before!) for each time stamp. The function takes three arguments:
1. data cube (the raw data the needs to be pre-loaded by servers to run the computations). One can drop spatial_extent from the definition of the data cube. However, if you know that all of your areas of interest (=rectangles surrounding the plumes) are located in some bounding box (e.g., you only want to study the plumes over the EU), then specifying that bounding box in spatial extent will speed up the computations.
2. set of areas of interest
3. reducer - a function, that is going to be applied to each area of interest at a given timestamp. In our case, a timestamp corresponds to a day and we are going to be computing the means. I.e, for rectangle A on Day B we compute mean_{all pixels in A}(value at pixel).

**1. Define the datacube**

In [15]:
s5_CH4 = connection.load_collection(
    "SENTINEL_5P_L2",
    #spatial_extent=optional_bounding_box_for_all_plume_rectangles,
    temporal_extent = period,
    bands=["CH4"] #as before, server-side computations for S5 accept only one band at a time
)

In [16]:
s3_all_bands = connection.load_collection(
    "SENTINEL3_SLSTR",
    #spatial_extent=optional_bounding_box_for_all_plume_rectangles,
    temporal_extent = period,
    bands=["S5", "S6"] #as before, server-side computations for S3 accept only several bands at a time
)

**2. Pass the set of areas of interest**

We are going to be using the aggregate_spatial function in OpenEO to compute the statistics for several areas at one.

The information about the areas of interest (in our case, 0.5 x 0.5 deg polygons centered at the plume coordinates), need to be passed using the GeoJSON format. You can use the template below and just change the coordinates. Note:
- Usually, one gives coordinates in the format (lat, lon) but in GeoJSON the order is reversed, so you need to write (lon, lat)
- A rectangle in GeoJSON has 5 vertices: the first and the last should be the same (you need to close the polygon). It is important that the format of the first and last vertex is exactly the same.
- The vertices need to be entered in counter-clockwise order starting from the South-Eastern vertex and have the form 
"coordinates":[[[max,min],[min,min],[min,max],[max,max],[max,min]]]

For example, consider the plume from coal site with coordinates [36.75,	109.76] (this is the 1st plume in [all TROPOMI detected plumes for 2021. (Schuit et al. 2023)](https://zenodo.org/records/8087134)). The coordinates of the rectangle are

"coordinates":[[[110.01,36.5],[109.51,36.5],[109.51,37.0 ],[110.01,37.0],[110.01,36.5]]]

In [17]:
# List of areas of interest used as an argument in aggregate_spatial. The coordinates correspond to the 0.5 x 0.5 deg rectangles 
# around the first two plumes from coal sites (plume [36.75, 109.76] and plume [37.53, 110.75]) in the dataset.
features = {"type": "FeatureCollection", "features": [
    {
        "type": "Feature", "properties": {},
        "geometry": {"type": "Polygon", "coordinates": [[
            [110.01, 36.5],[109.51, 36.5],[109.51, 37.0],[110.01, 37.0],[110.01, 36.5]
        ]]}
    },
    {
        "type": "Feature", "properties": {},
        "geometry": {"type": "Polygon", "coordinates": [[
            [111.0, 37.28],[110.5, 37.28],[110.5, 37.78],[111.0, 37.78],[111.0, 37.28]
        ]]}
    }
]}

**3. Add reducer and initiate the job**

In [18]:
s5_CH4_aggregation = s5_CH4.aggregate_spatial(
    geometries=features,
    reducer="mean",
)
s5_CH4_aggregation = s5_CH4_aggregation.save_result(format="JSON")
job=s5_CH4_aggregation.create_job(title="s5_CH4_aggregation")
job.start_and_wait()

0:00:00 Job 'j-240212264b5145b784aea23bd501dc23': send 'start'
0:00:13 Job 'j-240212264b5145b784aea23bd501dc23': queued (progress N/A)
0:00:19 Job 'j-240212264b5145b784aea23bd501dc23': queued (progress N/A)
0:00:35 Job 'j-240212264b5145b784aea23bd501dc23': queued (progress N/A)
0:00:43 Job 'j-240212264b5145b784aea23bd501dc23': queued (progress N/A)
0:00:53 Job 'j-240212264b5145b784aea23bd501dc23': queued (progress N/A)
0:01:06 Job 'j-240212264b5145b784aea23bd501dc23': queued (progress N/A)
0:01:22 Job 'j-240212264b5145b784aea23bd501dc23': running (progress N/A)
0:01:41 Job 'j-240212264b5145b784aea23bd501dc23': running (progress N/A)
0:02:06 Job 'j-240212264b5145b784aea23bd501dc23': running (progress N/A)
0:02:42 Job 'j-240212264b5145b784aea23bd501dc23': finished (progress N/A)


<BatchJob job_id='j-240212264b5145b784aea23bd501dc23'>

In [19]:
job

<BatchJob job_id='j-240212264b5145b784aea23bd501dc23'>

In [20]:
results = job.get_results()
results.download_file("output/s5_CH4_aggregation.json")

PosixPath('s5_CH4_aggregation.json')

In [21]:
s3_aggregation = s3_all_bands.aggregate_spatial(
    geometries=features,
    reducer="mean",
)
s3_aggregation = s3_aggregation.save_result(format="JSON")
job=s3_aggregation.create_job(title="s3_aggregation")
job.start_and_wait()

0:00:00 Job 'j-24021281012d4a91a03d70317b1c6be1': send 'start'
0:00:17 Job 'j-24021281012d4a91a03d70317b1c6be1': created (progress N/A)
0:00:23 Job 'j-24021281012d4a91a03d70317b1c6be1': running (progress N/A)
0:00:31 Job 'j-24021281012d4a91a03d70317b1c6be1': running (progress N/A)
0:00:41 Job 'j-24021281012d4a91a03d70317b1c6be1': running (progress N/A)
0:00:56 Job 'j-24021281012d4a91a03d70317b1c6be1': running (progress N/A)
0:01:10 Job 'j-24021281012d4a91a03d70317b1c6be1': running (progress N/A)
0:01:25 Job 'j-24021281012d4a91a03d70317b1c6be1': running (progress N/A)
0:01:45 Job 'j-24021281012d4a91a03d70317b1c6be1': running (progress N/A)
0:02:10 Job 'j-24021281012d4a91a03d70317b1c6be1': running (progress N/A)
0:02:40 Job 'j-24021281012d4a91a03d70317b1c6be1': finished (progress N/A)


<BatchJob job_id='j-24021281012d4a91a03d70317b1c6be1'>

In [22]:
results = job.get_results()
results.download_file("output/s3_aggregation.json")

PosixPath('output/s3_aggregation.json')

## From JSON to timeseries dataframe

The openEO Python Client Library has a helper function that converts a JSON into pandas dataframe. From there, you can compute column-wise statistics (min, max, mean, sd and median)

In [1]:
import json
import pandas as pd
from openeo.rest.conversions import timeseries_json_to_pandas

with open("output/s5_CH4_aggregation.json") as f:
    data = json.load(f)

df_s5 = timeseries_json_to_pandas(data)
df_s5.index = pd.to_datetime(df_s5.index)

print(len(df_s5))
df_s5.head()

24


polygon,0,1
date,,
2022-05-01 00:00:00+00:00,1871.712693,1891.123617
2022-05-02 00:00:00+00:00,NaN,1894.057792
2022-05-03 00:00:00+00:00,1878.450416,1892.246428
2022-05-04 00:00:00+00:00,1886.297213,1873.674883
2022-05-05 00:00:00+00:00,1881.929169,NaN


In [2]:
with open("output/s3_aggregation.json") as f:
    data = json.load(f)

df_s3 = timeseries_json_to_pandas(data)
df_s3.index = pd.to_datetime(df_s3.index)

print(len(df_s3))
df_s3.head()

28


polygon                           0                   1          
band                              0         1         0         1
date                                                             
2022-05-02 00:00:00+00:00 -0.000054  0.002344 -0.000025  0.001993
2022-05-03 00:00:00+00:00 -0.000064  0.003103  0.000264  0.003348
2022-05-04 00:00:00+00:00  0.000245  0.002510  0.000540  0.003280
2022-05-05 00:00:00+00:00  0.000207  0.002211  0.000460  0.001994
2022-05-06 00:00:00+00:00  0.000865  0.004442  0.000087  0.003695

## Divide and conquer

Note that the csv file contains the locations of 1913 plumes that come from oil, gas or coal locations. This is quite a bit of loading per user, and Copernicus has user quotas, which are pretty cryptic, so I cannot guarantee that a batch job of all computations for all plumes would not run out of quota. Instead, you are encouraged to divide and conquer: either with your friends and/or by creating several user profiles on Copernicus. Some tips:
- you can re-connect to a job at another time, by another script/notebook or even with another openEO client within 7 days;
- Copernicus servers can process up to 2 batch jobs per user simultaneously;
- Copernicus has certain usage quotas. To get a sense of where you are at the moment, you can check you status using [Sentinel Hub Services Dashboard](https://shapps.dataspace.copernicus.eu/dashboard/#/). Also, running the 'job' command after a job is finished will tell you have many credits were used.

**Minimal requirement for passing assignment 3:** the SVM needs to be performed on at least 50 plumes; one student should load the data for at least 20 plumes.

## Increase batch resources

In principle, you can also increase the computational resources available for your batch jobs, as shown in [these instructions](https://documentation.dataspace.copernicus.eu/APIs/openEO/job_config.html). Some warnings:
- I do not know how far these can be pushed, but you could use the options for server settings when you try to access Copernicus JupyterLab [here](https://dataspace.copernicus.eu/analyse/jupyterlab)
- My understanding is that a substantial portion of the total job execution time is the queue waiting time